In [4]:
# get length of all csv files in a directory

import os
import csv
import pandas as pd

def get_csv_length(file):
    df = pd.read_csv(f'../data/csv/{file}')
    return len(df)/32
    
directory = "../data/csv"
files = [f for f in os.listdir(directory) if f.endswith('.csv')]
for file in files:
    print(f'{file}: {get_csv_length(file)}')

FT_32x6mm-airwater-eps35.csv: 481.0
FT_32x6mm-airwater-eps25.csv: 481.0
FT_32x4mm-airwater-eps15.csv: 275.0
FT_32x4mm-airwater-eps25.csv: 454.0
FT_32x4mm-airwater-eps35.csv: 481.0
FT_32x6mm-airwater-eps15.csv: 296.0


In [1]:
import pandas as pd
from scripts.dataset import (
    process_csv,
    compute_box_size,
    apply_periodic_boundary_conditions,
    check_trajectory,
)
import torch
import pandas as pd
import os
import torch
from torch_geometric.data import Data, InMemoryDataset
from tqdm import tqdm, trange

# from scripts.models import EGNN_vel, MSE_harmonics, Trainer

# Load the CSV file into a DataFrame
df = pd.read_csv("../data/pickles_csv/FT_32x4mm-airwater-eps15.csv")

# Display the first few rows of the DataFrame
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/pickles_csv/FT_32x4mm-airwater-eps15.csv'

In [2]:
df_filtered = df[df["bub_num"] == 0]
df_filtered = df_filtered[["bub_num", "time [s]", "vel_x", "vel_y", "vel_z"]]
df_filtered.iloc[0:20]

,bub_num,time [s],vel_x,vel_y,vel_z
0,0,0.20,-0.074537,0.079467,0.138480
32,0,0.21,-0.111090,0.051433,0.153936
64,0,0.22,-0.020208,0.058066,0.060245
96,0,0.23,-0.013191,0.059969,0.065174
128,0,0.24,0.036984,0.017956,0.089404
160,0,0.25,0.025277,0.005854,0.185379
192,0,0.26,-0.034197,0.031990,0.157432
224,0,0.27,-0.003539,0.053398,0.192336
256,0,0.28,0.001879,0.017664,0.171606
288,0,0.29,-0.054629,0.015735,0.148776


In [3]:
min_orb_values, max_orb_values = process_csv("../data/pickles_csv/", "../data/raw/")

Processing file:  FT_32x6mm-airwater-eps35.csv
Box size:  0.021786030258038356
Processing file:  FT_32x6mm-airwater-eps25.csv
Box size:  0.024371791141518228
Processing file:  FT_32x4mm-airwater-eps15.csv
Box size:  0.019263969051043647
Processing file:  FT_32x4mm-airwater-eps25.csv
Box size:  0.016247860761012152
Processing file:  FT_32x4mm-airwater-eps35.csv
Box size:  0.014524020172025571
Processing file:  FT_32x6mm-airwater-eps15.csv
Box size:  0.028895953576565468


In [4]:
class BubbleDataset(InMemoryDataset):
    def __init__(
        self,
        raw_data_path,
        processed_data_path,
        root=None,
        processed_file_name="BubbleDataset_1.pt",
        num_bubbles=32,
        min_orb_values=None,
        max_orb_values=None,
    ):
        self.raw_data_path = raw_data_path
        self.processed_data_path = processed_data_path
        self.processed_file_name = processed_file_name
        self.num_bubbles = num_bubbles
        if min_orb_values is not None:
            self.min_orb_values = torch.tensor(min_orb_values, dtype=torch.float)
        else:
            self.min_orb_values = None
        if max_orb_values is not None:
            self.max_orb_values = torch.tensor(max_orb_values, dtype=torch.float)
        else:
            self.max_orb_values = None
        super(BubbleDataset, self).__init__(root)

        # Define processed file path
        processed_file_path = self.processed_paths[0]

        # Check if the processed file exists; if not, call process() to generate it
        if not os.path.exists(self.processed_paths[0]):
            print(
                f"Processed file not found at {self.processed_paths[0]}. Generating..."
            )
            self.process()
        else:
            # Load the processed data
            self.data, self.slices = torch.load(processed_file_path)

    @property
    def raw_file_names(self):
        return [fn for fn in os.listdir(self.raw_data_path)]

    @property
    def processed_paths(self):
        return [os.path.join(self.processed_data_path, self.processed_file_name)]

    def download(self):
        pass

    def __len__(self):

        total_time_steps = 0
        for idx, raw_path in enumerate(self.raw_file_names):

            # Load trajectory from pickle file
            trajectory = pd.read_pickle(self.raw_data_path + raw_path)

            # Count the number of unique time steps
            num_time_steps = len(trajectory["time [s]"].unique())
            total_time_steps += (
                num_time_steps - 1
            )  # Subtract 1 because we need a previous and next time step

        return total_time_steps

    def process(self):
        data_list = []
        for idx, raw_path in enumerate(self.raw_file_names):

            # Load trajectory from pickle file
            try:

                trajectory = pd.read_pickle(self.raw_data_path + raw_path)
            except Exception as e:
                print(
                    f"Errore nel caricamento del file {self.raw_data_path + raw_path}: {e}"
                )

            # Ensure data is sorted by time and bubble number for correct indexing
            trajectory = trajectory.sort_values(by=["time [s]", "bub_num"]).reset_index(
                drop=True
            )

            # Compute the scaling factor for the trajectory
            r_bub = trajectory["r_bub"].values[0]
            eps = trajectory["eps"].values[0]
            box_size = compute_box_size(r_bub, eps, self.num_bubbles)

            # Every attribute that starts with orb_ is an orbital attribute
            # Select columns that start with "orb_"
            orb_columns = [col for col in trajectory.columns if col.startswith("orb_")]

            time_steps = trajectory["time [s]"].unique()

            for i in range(len(time_steps) - 1):
                # Get the time corresponding to the current index
                current_time = time_steps[i]
                next_time = time_steps[i + 1]

                # Filter the data for the current and next time steps
                current_data = trajectory[trajectory["time [s]"] == current_time]
                next_data = trajectory[trajectory["time [s]"] == next_time]

                # Extract the positions and velocities of the current step
                current_pos = torch.tensor(
                    current_data[["pos_x", "pos_y", "pos_z"]].values,
                    dtype=torch.float,
                    requires_grad=True,
                )
                current_vel = torch.tensor(
                    current_data[["vel_x", "vel_y", "vel_z"]].values,
                    dtype=torch.float,
                    requires_grad=True,
                )

                # Extract the positions and velocities of the next step
                next_pos = torch.tensor(
                    next_data[["pos_x", "pos_y", "pos_z"]].values, dtype=torch.float
                )
                next_vel = torch.tensor(
                    next_data[["vel_x", "vel_y", "vel_z"]].values,
                    dtype=torch.float,
                )

                # Scale positions
                current_pos = current_pos / box_size
                next_pos = next_pos / box_size

                # Check that all positions
                assert torch.all(current_pos >= 0) and torch.all(current_pos <= 1)
                assert torch.all(next_pos >= 0) and torch.all(next_pos <= 1)

                # Convert the selected columns to a tensor
                current_orbs = torch.tensor(
                    current_data[orb_columns].values,
                    dtype=torch.float,
                )
                next_orbs = torch.tensor(
                    next_data[orb_columns].values, dtype=torch.float
                )

                # Normalize the orbital attributes
                if self.min_orb_values is not None and self.max_orb_values is not None:

                    current_orbs = (current_orbs - self.min_orb_values) / (
                        self.max_orb_values - self.min_orb_values
                    )

                    next_orbs = (next_orbs - self.min_orb_values) / (
                        self.max_orb_values - self.min_orb_values
                    )
                    # Check that all orbital attributes are within the range [0, 1]
                    assert torch.all(current_orbs >= 0) and torch.all(current_orbs <= 1)
                    assert torch.all(next_orbs >= 0) and torch.all(next_orbs <= 1)
                else:
                    print(
                        "WARNING: Min and max values for the orbital attributes are not provided. "
                        "Orbital attributes are not normalized."
                    )

                current_orbs.requires_grad_()

                # TODO add eps value

                # Create edge index and edge attributes
                edge_index = (
                    torch.tensor(
                        [
                            [i, j]
                            for i in range(self.num_bubbles)
                            for j in range(self.num_bubbles)
                            if i != j
                        ],
                        dtype=torch.long,
                    )
                    .t()
                    .contiguous()
                )

                edge_attr = torch.stack(
                    [
                        torch.cat(
                            (
                                current_pos[j] - current_pos[i],
                                current_vel[j] - current_vel[i],
                            )
                        )
                        for i in range(self.num_bubbles)
                        for j in range(self.num_bubbles)
                        if i != j
                    ],
                    dim=0,
                )

                data = Data(
                    x=(current_pos, current_vel, current_orbs),
                    y=(next_pos, next_vel, next_orbs),
                    edge_index=edge_index,
                    edge_attr=edge_attr,
                    num_nodes=self.num_bubbles,
                )
                data_list.append(data)

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_data_path + self.processed_file_name)

    def __getitem__(self, idx):
        return self.get(idx)

    def __repr__(self):
        return f"{self.__class__.__name__}({len(self)})"

In [5]:
bubble_dataset = BubbleDataset(
    "../data/raw/",
    "../data/processed/",
    root="../data/",
    processed_file_name="BubbleDataset_debug.pt",
    num_bubbles=32,
    min_orb_values=min_orb_values,
    max_orb_values=max_orb_values,
)

Processing...
Done!
/var/folders/rb/wmrnk95s19s54kfvkb8zgwcm0000gn/T/ipykernel_10893/1335299690.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices =

In [6]:
import os

csv_directory = "../data/pickles_csv"
total_lines = 0

for filename in os.listdir(csv_directory):
    if filename.endswith(".csv"):
        with open(os.path.join(csv_directory, filename), "r") as file:
            line_count = sum(1 for line in file) - 1
            if line_count % 32 != 0:
                print(
                    f"File {filename} has {line_count} lines, which is not a multiple of 32"
                )
            print(f"{filename}: {line_count/32} time steps")
            total_lines += line_count

print(f"Total lines in all CSV files: {total_lines/32}")

FT_32x6mm-airwater-eps35.csv: 481.0 time steps
FT_32x6mm-airwater-eps25.csv: 481.0 time steps
FT_32x4mm-airwater-eps15.csv: 275.0 time steps
FT_32x4mm-airwater-eps25.csv: 454.0 time steps
FT_32x4mm-airwater-eps35.csv: 481.0 time steps
FT_32x6mm-airwater-eps15.csv: 296.0 time steps
Total lines in all CSV files: 2468.0


In [7]:
total_lines / 32 * 0.2 / 6

82.26666666666667

In [8]:
from scripts.dataset import compute_box_size

compute_box_size(0.003, 0.15, 32)

0.028895953576565468

In [ ]:
for file in os.listdir("../data/csv"):
    if file.endswith(".csv"):
        df = pd.read_csv(f"../data/csv/{file}")
        # Get max and min velocity
        max_vel_x = df["vel_x"].max()
        min_vel_x = df["vel_x"].min()
        max_vel_y = df["vel_y"].max()
        min_vel_y = df["vel_y"].min()
        max_vel_z = df["vel_z"].max()
        min_vel_z = df["vel_z"].min()

        # print(
        #     f"max_vel_x: {max_vel_x}\t min_vel_x: {min_vel_x}"
        # )
        # print(
        #     f"max_vel_y: {max_vel_y}\t min_vel_y: {min_vel_y}\n "
        # )
        # print(
        #     f"max_vel_z: {max_vel_z}\t min_vel_z: {min_vel_z}"
        # )
        print(f"File: {file}")
        orb_columns = [col for col in df.columns if col.startswith("orb_")][:16]

        for col in orb_columns:
            # Count the number of lines out of range [-1, 1], and print index
            # out_of_range = df[(df[col] < -1) | (df[col] > 1)]
            # if len(out_of_range) > 0:
            #     print(f"Column: {col}")
            #     print(out_of_range.index)
            #     print("\n")
            # Find min and max values
            max_value = df[col].max()
            min_value = df[col].min()
            print(f"Column: {col}")
            print(f"max: {max_value}\t min: {min_value}")

File: FT_32x6mm-airwater-eps35.csv
Column: orb_0
max: 0.010590417817804	 min: 0.0082845874533349
Column: orb_1
max: 0.0019562867934808	 min: -0.00170227462451
Column: orb_2
max: 0.0022501847542276	 min: -0.0028906831354665
Column: orb_3
max: 0.0017873784900572	 min: -0.0021125841497128
Column: orb_4
max: 0.0019256514430752	 min: -0.0016271965598415
Column: orb_5
max: 0.0026889150815153	 min: -0.002395151912125
Column: orb_6
max: 0.001189498639316	 min: -0.0036581170159488
Column: orb_7
max: 0.0023915939819514	 min: -0.0024206629774622
Column: orb_8
max: 0.0020228274799359	 min: -0.0016821279259392
Column: orb_9
max: 0.0006243575592163	 min: -0.0006865121513914
Column: orb_10
max: 0.0008583976574868	 min: -0.0009544511042719
Column: orb_11
max: 0.0008264594246529	 min: -0.0008933761387753
Column: orb_12
max: 0.0011307730041211	 min: -0.0009792069244518
Column: orb_13
max: 0.0008430471849793	 min: -0.0006872633328726
Column: orb_14
max: 0.0009324990963879	 min: -0.0008648949697376
Column

In [ ]:
df = pd.read_csv("../data/csv/FT_32x6mm-airwater-eps15.csv")


df_filtered = df[df["orb_0"] < 0]

df_filtered = df_filtered[df_filtered["orb_0"] > 1]
df_filtered.head()

,Unnamed: 0,id,sim,bub_num,time [s],pos_x,pos_y,pos_z,vel_x,vel_y,...,orb_215,orb_216,orb_217,orb_218,orb_219,orb_220,orb_221,orb_222,orb_223,orb_224
